百度网盘AI大赛-文档检测优化赛第4名方案



定义dataloader



In [ ]:
import paddle

import numpy as np
import pandas as pd

import cv2
import matplotlib.pyplot as plt
import albumentations as A

def train_augmentation():
    train_transform = [
        # A.RandomSizedCrop(min_max_height=(1024, 1024), height=1024, width=1024, w2h_ratio=1.0, always_apply=False, p=1.),
        A.Flip(p=0.5),
        A.Rotate(p=0.5),
        ############################ add #################
        # A.RandomGridShuffle(grid=(2, 2), p=0.2),
        #
        # A.Transpose(p=0.5),
        # A.ShiftScaleRotate(p=0.5),
        ############################ add #################

        # A.OneOf([
        #     A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=1.),
        #     A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, p=1.),
        #     A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=1.),
        # ], p=0.5),

    ]
    return A.Compose(train_transform)


class MyDateset(paddle.io.Dataset):
    def __init__(self, mode='train', train_imgs_dir='D:/yyc/competition/AIstudio/Document_detection/train_datasets_document_detection_0411/images/', transform=train_augmentation(),
                 label_imgs_dir='D:/yyc/competition/AIstudio/Document_detection/train_datasets_document_detection_0411/segments/', train_txt='D:/yyc/competition/AIstudio/Document_detection/train_datasets_document_detection_0411/data_info.txt'):
        super(MyDateset, self).__init__()

        self.mode = mode
        self.train_imgs_dir = train_imgs_dir
        self.label_imgs_dir = label_imgs_dir

        self.transform = transform

        with open(train_txt, 'r') as f:
            self.train_infor = f.readlines()


    def __getitem__(self, index):
        item = self.train_infor[index][:-1]
        splited = item.split(',')
        img_name = splited[0]

        img = cv2.imread(self.train_imgs_dir + img_name + '.jpg')
        h, w, c = img.shape

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        lab = cv2.imread(self.label_imgs_dir + img_name + '.png')

        trans = self.transform(image=img, mask=lab)
        img, lab = trans['image'], trans['mask']


        lab = paddle.vision.transforms.resize(lab, (768, 768), interpolation='nearest')
        lab = (lab[:, :, 0] == 255)

        # plt.imshow(lab)
        # plt.show()


        # 对图片进行resize，调整明暗对比度等参数
        img = paddle.vision.transforms.resize(img, (768, 768), interpolation='bilinear')
        if np.random.rand() < 1 / 2:
            img = paddle.vision.transforms.adjust_brightness(img, np.random.rand() * 2)
        else:
            if np.random.rand() < 1 / 2:
                img = paddle.vision.transforms.adjust_contrast(img, np.random.rand() * 2)
            else:
                img = paddle.vision.transforms.adjust_hue(img, np.random.rand() - 0.5)



        img = img.transpose((2, 0, 1))
        img = img / 255



        img = paddle.to_tensor(img).astype('float32')
        label = paddle.to_tensor(lab).astype('int64')

        return img, label


    def __len__(self):
        return len(self.train_infor)



定义model

In [ ]:
import os
import paddle
import paddle.nn as nn
import paddle.nn.functional as F
import numpy as np
import paddle.nn.initializer as paddle_init


# from paddleseg.cvlibs import manager
# from paddleseg.utils import utils
# from paddleseg.models.backbones.transformer_utils import *


def load_pretrained_model(model, pretrained_model):
    if pretrained_model is not None:
        print('Loading pretrained model from {}'.format(pretrained_model))

        # if urlparse(pretrained_model).netloc:
        #     pretrained_model = download_pretrained_model(pretrained_model)

        if os.path.exists(pretrained_model):
            para_state_dict = paddle.load(pretrained_model)

            model_state_dict = model.state_dict()
            keys = model_state_dict.keys()
            num_params_loaded = 0
            for k in keys:
                if k not in para_state_dict:
                    print("{} is not in pretrained model".format(k))
                elif list(para_state_dict[k].shape) != list(model_state_dict[k]
                                                            .shape):
                    print(
                        "[SKIP] Shape of pretrained params {} doesn't match.(Pretrained: {}, Actual: {})"
                        .format(k, para_state_dict[k].shape, model_state_dict[k]
                                .shape))
                else:
                    model_state_dict[k] = para_state_dict[k]
                    num_params_loaded += 1
            model.set_dict(model_state_dict)
            print("There are {}/{} variables loaded into {}.".format(
                num_params_loaded,
                len(model_state_dict), model.__class__.__name__))

        else:
            raise ValueError('The pretrained model directory is not Found: {}'.
                             format(pretrained_model))
    else:
        print(
            'No pretrained model to load, {} will be trained from scratch.'.
            format(model.__class__.__name__))



def to_2tuple(x):
    return tuple([x] * 2)


def drop_path(x, drop_prob=0., training=False):
    """Drop paths (Stochastic Depth) per sample (when applied in main path of residual blocks).
    the original name is misleading as 'Drop Connect' is a different form of dropout in a separate paper...
    See discussion: https://github.com/tensorflow/tpu/issues/494#issuecomment-532968956 ...
    """
    if drop_prob == 0. or not training:
        return x
    keep_prob = paddle.to_tensor(1 - drop_prob)
    shape = (paddle.shape(x)[0], ) + (1, ) * (x.ndim - 1)
    random_tensor = keep_prob + paddle.rand(shape, dtype=x.dtype)
    random_tensor = paddle.floor(random_tensor)  # binarize
    output = x.divide(keep_prob) * random_tensor
    return output


class DropPath(nn.Layer):
    """Drop paths (Stochastic Depth) per sample  (when applied in main path of residual blocks).
    """

    def __init__(self, drop_prob=None):
        super(DropPath, self).__init__()
        self.drop_prob = drop_prob

    def forward(self, x):
        return drop_path(x, self.drop_prob, self.training)


class Identity(nn.Layer):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, input):
        return input


trunc_normal_ = paddle_init.TruncatedNormal(std=.02)
zeros_ = paddle_init.Constant(value=0.)
ones_ = paddle_init.Constant(value=1.)


def init_weights(layer):
    """
    Init the weights of transformer.
    Args:
        layer(nn.Layer): The layer to init weights.
    Returns:
        None
    """
    if isinstance(layer, nn.Linear):
        trunc_normal_(layer.weight)
        if layer.bias is not None:
            zeros_(layer.bias)
    elif isinstance(layer, nn.LayerNorm):
        zeros_(layer.bias)
        ones_(layer.weight)


class Mlp(nn.Layer):
    """ Multilayer perceptron."""

    def __init__(self,
                 in_features,
                 hidden_features=None,
                 out_features=None,
                 act_layer=nn.GELU,
                 drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x


def window_partition(x, window_size):
    """
    Args:
        x: (B, H, W, C)
        window_size (int): window size
    Returns:
        windows: (num_windows*B, window_size, window_size, C)
    """
    B, H, W, C = x.shape
    x = x.reshape(
        [B, H // window_size, window_size, W // window_size, window_size, C])
    windows = x.transpose([0, 1, 3, 2, 4,
                           5]).reshape([-1, window_size, window_size, C])
    return windows


def window_reverse(windows, window_size, H, W):
    """
    Args:
        windows: (num_windows*B, window_size, window_size, C)
        window_size (int): Window size
        H (int): Height of image
        W (int): Width of image
    Returns:
        x: (B, H, W, C)
    """
    B = int(windows.shape[0] / (H * W / window_size / window_size))
    x = windows.reshape(
        [B, H // window_size, W // window_size, window_size, window_size, -1])
    x = x.transpose([0, 1, 3, 2, 4, 5]).reshape([B, H, W, -1])
    return x


class WindowAttention(nn.Layer):
    """
    Window based multi-head self attention (W-MSA) module with relative position bias.
    It supports both of shifted and non-shifted window.
    Args:
        dim (int): Number of input channels.
        window_size (tuple[int]): The height and width of the window.
        num_heads (int): Number of attention heads.
        qkv_bias (bool, optional):  If True, add a learnable bias to query, key, value. Default: True
        qk_scale (float | None, optional): Override default qk scale of head_dim ** -0.5 if set
        attn_drop (float, optional): Dropout ratio of attention weight. Default: 0.0
        proj_drop (float, optional): Dropout ratio of output. Default: 0.0
    """

    def __init__(self,
                 dim,
                 window_size,
                 num_heads,
                 qkv_bias=True,
                 qk_scale=None,
                 attn_drop=0.,
                 proj_drop=0.):

        super().__init__()
        self.dim = dim
        self.window_size = window_size  # Wh, Ww
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim**-0.5

        # define a parameter table of relative position bias
        self.relative_position_bias_table = self.create_parameter(
            shape=((2 * window_size[0] - 1) * (2 * window_size[1] - 1),
                   num_heads),
            default_initializer=zeros_)
        self.add_parameter("relative_position_bias_table",
                           self.relative_position_bias_table)

        # get pair-wise relative position index for each token inside the window
        coords_h = paddle.arange(self.window_size[0])
        coords_w = paddle.arange(self.window_size[1])
        coords = paddle.stack(paddle.meshgrid([coords_h,
                                               coords_w]))  # 2, Wh, Ww
        coords_flatten = paddle.flatten(coords, 1)  # 2, Wh*Ww
        coords_flatten_1 = coords_flatten.unsqueeze(axis=2)
        coords_flatten_2 = coords_flatten.unsqueeze(axis=1)
        relative_coords = coords_flatten_1 - coords_flatten_2

        relative_coords = relative_coords.transpose([1, 2, 0])

        relative_coords[:, :, 0] += self.window_size[
            0] - 1  # shift to start from 0
        relative_coords[:, :, 1] += self.window_size[1] - 1
        relative_coords[:, :, 0] *= 2 * self.window_size[1] - 1
        relative_position_index = relative_coords.sum(-1)  # Wh*Ww, Wh*Ww
        self.register_buffer("relative_position_index", relative_position_index)

        self.qkv = nn.Linear(dim, dim * 3, bias_attr=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

        trunc_normal_(self.relative_position_bias_table)
        self.softmax = nn.Softmax(axis=-1)

    def forward(self, x, mask=None):
        """
        Args:
            x: input features with shape of (num_windows*B, N, C)
            mask: (0/-inf) mask with shape of (num_windows, Wh*Ww, Wh*Ww) or None
        """
        B_, N, C = x.shape
        qkv = self.qkv(x).reshape(
            [B_, N, 3, self.num_heads,
             C // self.num_heads]).transpose([2, 0, 3, 1, 4])
        q, k, v = qkv[0], qkv[1], qkv[2]

        q = q * self.scale
        attn = paddle.mm(q, k.transpose([0, 1, 3, 2]))

        index = self.relative_position_index.reshape([-1])
        relative_position_bias = paddle.index_select(
            self.relative_position_bias_table, index)

        relative_position_bias = relative_position_bias.reshape([
            self.window_size[0] * self.window_size[1],
            self.window_size[0] * self.window_size[1], -1
        ])  # Wh*Ww,Wh*Ww,nH
        relative_position_bias = relative_position_bias.transpose(
            [2, 0, 1])  # nH, Wh*Ww, Wh*Ww
        attn = attn + relative_position_bias.unsqueeze(0)

        if mask is not None:
            nW = mask.shape[0]
            attn = attn.reshape([B_ // nW, nW, self.num_heads, N, N
                                 ]) + mask.unsqueeze(1).unsqueeze(0)
            attn = attn.reshape([-1, self.num_heads, N, N])
            attn = self.softmax(attn)
        else:
            attn = self.softmax(attn)

        attn = self.attn_drop(attn)

        x = paddle.mm(attn, v).transpose([0, 2, 1, 3]).reshape([B_, N, C])
        x = self.proj(x)
        x = self.proj_drop(x)
        return x


class SwinTransformerBlock(nn.Layer):
    """
    Swin Transformer Block.
    Args:
        dim (int): Number of input channels.
        num_heads (int): Number of attention heads.
        window_size (int): Window size.
        shift_size (int): Shift size for SW-MSA.
        mlp_ratio (float): Ratio of mlp hidden dim to embedding dim.
        qkv_bias (bool, optional): If True, add a learnable bias to query, key, value. Default: True
        qk_scale (float | None, optional): Override default qk scale of head_dim ** -0.5 if set.
        drop (float, optional): Dropout rate. Default: 0.0
        attn_drop (float, optional): Attention dropout rate. Default: 0.0
        drop_path (float, optional): Stochastic depth rate. Default: 0.0
        act_layer (nn.Layer, optional): Activation layer. Default: nn.GELU
        norm_layer (nn.Layer, optional): Normalization layer.  Default: nn.LayerNorm
    """

    def __init__(self,
                 dim,
                 num_heads,
                 window_size=7,
                 shift_size=0,
                 mlp_ratio=4.,
                 qkv_bias=True,
                 qk_scale=None,
                 drop=0.,
                 attn_drop=0.,
                 drop_path=0.,
                 act_layer=nn.GELU,
                 norm_layer=nn.LayerNorm):
        super().__init__()
        self.dim = dim
        self.num_heads = num_heads
        self.window_size = window_size
        self.shift_size = shift_size
        self.mlp_ratio = mlp_ratio
        assert 0 <= self.shift_size < self.window_size, "shift_size must in 0-window_size"

        self.norm1 = norm_layer(dim)
        self.attn = WindowAttention(
            dim,
            window_size=to_2tuple(self.window_size),
            num_heads=num_heads,
            qkv_bias=qkv_bias,
            qk_scale=qk_scale,
            attn_drop=attn_drop,
            proj_drop=drop)

        self.drop_path = DropPath(drop_path) if drop_path > 0. else Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim,
                       hidden_features=mlp_hidden_dim,
                       act_layer=act_layer,
                       drop=drop)

        self.H = None
        self.W = None

    def forward(self, x, mask_matrix):
        """
        Args:
            x: Input feature, tensor size (B, H*W, C).
            H, W: Spatial resolution of the input feature.
            mask_matrix: Attention mask for cyclic shift.
        """
        B, L, C = x.shape
        H, W = self.H, self.W
        assert L == H * W, "input feature has wrong size"

        shortcut = x
        x = self.norm1(x)
        x = x.reshape([B, H, W, C])

        # pad feature maps to multiples of window size
        pad_l = pad_t = 0
        pad_r = (self.window_size - W % self.window_size) % self.window_size
        pad_b = (self.window_size - H % self.window_size) % self.window_size

        x = x.transpose([0, 3, 1, 2])
        x = F.pad(x, [pad_l, pad_r, pad_t, pad_b])
        x = x.transpose([0, 2, 3, 1])
        _, Hp, Wp, _ = x.shape

        # cyclic shift
        if self.shift_size > 0:
            shifted_x = paddle.roll(
                x, shifts=(-self.shift_size, -self.shift_size), axis=(1, 2))
            attn_mask = mask_matrix
        else:
            shifted_x = x
            attn_mask = None

        # partition windows
        x_windows = window_partition(
            shifted_x, self.window_size)  # nW*B, window_size, window_size, C
        x_windows = x_windows.reshape(
            [-1, self.window_size * self.window_size,
             C])  # nW*B, window_size*window_size, C

        # W-MSA/SW-MSA
        attn_windows = self.attn(
            x_windows, mask=attn_mask)  # nW*B, window_size*window_size, C

        # merge windows
        attn_windows = attn_windows.reshape(
            [-1, self.window_size, self.window_size, C])
        shifted_x = window_reverse(attn_windows, self.window_size, Hp,
                                   Wp)  # B H' W' C

        # reverse cyclic shift
        if self.shift_size > 0:
            x = paddle.roll(
                shifted_x,
                shifts=(self.shift_size, self.shift_size),
                axis=(1, 2))
        else:
            x = shifted_x

        if pad_r > 0 or pad_b > 0:
            x = x[:, :H, :W, :]

        x = x.reshape([B, H * W, C])

        # FFN
        x = shortcut + self.drop_path(x)
        x = x + self.drop_path(self.mlp(self.norm2(x)))

        return x


class PatchMerging(nn.Layer):
    """
    Patch Merging Layer
    Args:
        dim (int): Number of input channels.
        norm_layer (nn.Layer, optional): Normalization layer.  Default: nn.LayerNorm
    """

    def __init__(self, dim, norm_layer=nn.LayerNorm):
        super().__init__()
        self.dim = dim
        self.reduction = nn.Linear(4 * dim, 2 * dim, bias_attr=False)
        self.norm = norm_layer(4 * dim)

    def forward(self, x, H, W):
        """
        Args:
            x: Input feature, tensor size (B, H*W, C).
            H, W: Spatial resolution of the input feature.
        """
        B, L, C = x.shape
        assert L == H * W, "input feature has wrong size"

        x = x.reshape([B, H, W, C])

        # padding
        pad_input = (H % 2 == 1) or (W % 2 == 1)
        if pad_input:
            x = x.transpose([0, 3, 1, 2])
            x = F.pad(x, [0, W % 2, 0, H % 2])
            x = x.transpose([0, 2, 3, 1])

        x0 = x[:, 0::2, 0::2, :]  # B H/2 W/2 C
        x1 = x[:, 1::2, 0::2, :]  # B H/2 W/2 C
        x2 = x[:, 0::2, 1::2, :]  # B H/2 W/2 C
        x3 = x[:, 1::2, 1::2, :]  # B H/2 W/2 C
        x = paddle.concat([x0, x1, x2, x3], -1)  # B H/2 W/2 4*C
        x = x.reshape([B, -1, 4 * C])  # B H/2*W/2 4*C

        x = self.norm(x)
        x = self.reduction(x)

        return x


class BasicLayer(nn.Layer):
    """
    A basic Swin Transformer layer for one stage.
    Args:
        dim (int): Number of feature channels.
        depth (int): Depths of this stage.
        num_heads (int): Number of attention head.
        window_size (int): Local window size. Default: 7.
        mlp_ratio (float): Ratio of mlp hidden dim to embedding dim. Default: 4.
        qkv_bias (bool, optional): If True, add a learnable bias to query, key, value. Default: True
        qk_scale (float | None, optional): Override default qk scale of head_dim ** -0.5 if set.
        drop (float, optional): Dropout rate. Default: 0.0
        attn_drop (float, optional): Attention dropout rate. Default: 0.0
        drop_path (float | tuple[float], optional): Stochastic depth rate. Default: 0.0
        norm_layer (nn.Layer, optional): Normalization layer. Default: nn.LayerNorm
        downsample (nn.Layer | None, optional): Downsample layer at the end of the layer. Default: None
    """

    def __init__(self,
                 dim,
                 depth,
                 num_heads,
                 window_size=7,
                 mlp_ratio=4.,
                 qkv_bias=True,
                 qk_scale=None,
                 drop=0.,
                 attn_drop=0.,
                 drop_path=0.,
                 norm_layer=nn.LayerNorm,
                 downsample=None):
        super().__init__()
        self.window_size = window_size
        self.shift_size = window_size // 2
        self.depth = depth

        # build blocks
        self.blocks = nn.LayerList([
            SwinTransformerBlock(
                dim=dim,
                num_heads=num_heads,
                window_size=window_size,
                shift_size=0 if (i % 2 == 0) else window_size // 2,
                mlp_ratio=mlp_ratio,
                qkv_bias=qkv_bias,
                qk_scale=qk_scale,
                drop=drop,
                attn_drop=attn_drop,
                drop_path=drop_path[i]
                if isinstance(drop_path, list) else drop_path,
                norm_layer=norm_layer) for i in range(depth)
        ])

        # patch merging layer
        if downsample is not None:
            self.downsample = downsample(dim=dim, norm_layer=norm_layer)
        else:
            self.downsample = None

    def forward(self, x, H, W):
        """
        Args:
            x: Input feature, tensor size (B, H*W, C).
            H, W: Spatial resolution of the input feature.
        """
        # calculate attention mask for SW-MSA
        Hp = int(np.ceil(H / self.window_size)) * self.window_size
        Wp = int(np.ceil(W / self.window_size)) * self.window_size
        img_mask = paddle.zeros((1, Hp, Wp, 1))  # 1 Hp Wp 1
        h_slices = (slice(0, -self.window_size),
                    slice(-self.window_size, -self.shift_size),
                    slice(-self.shift_size, None))
        w_slices = (slice(0, -self.window_size),
                    slice(-self.window_size, -self.shift_size),
                    slice(-self.shift_size, None))
        cnt = 0
        for h in h_slices:
            for w in w_slices:
                img_mask[:, h, w, :] = cnt
                cnt += 1

        mask_windows = window_partition(
            img_mask, self.window_size)  # nW, window_size, window_size, 1
        mask_windows = mask_windows.reshape(
            [-1, self.window_size * self.window_size])
        attn_mask = mask_windows.unsqueeze(1) - mask_windows.unsqueeze(2)

        huns = -100.0 * paddle.ones_like(attn_mask)
        attn_mask = huns * (attn_mask != 0).astype("float32")

        for blk in self.blocks:
            blk.H, blk.W = H, W
            x = blk(x, attn_mask)
        if self.downsample is not None:
            x_down = self.downsample(x, H, W)
            Wh, Ww = (H + 1) // 2, (W + 1) // 2
            return x, H, W, x_down, Wh, Ww
        else:
            return x, H, W, x, H, W


class PatchEmbed(nn.Layer):
    """
    Image to Patch Embedding.
    Args:
        patch_size (int): Patch token size. Default: 4.
        in_chans (int): Number of input image channels. Default: 3.
        embed_dim (int): Number of linear projection output channels. Default: 96.
        norm_layer (nn.Layer, optional): Normalization layer. Default: None
    """

    def __init__(self, patch_size=4, in_chans=3, embed_dim=96, norm_layer=None):
        super().__init__()
        patch_size = to_2tuple(patch_size)
        self.patch_size = patch_size

        self.in_chans = in_chans
        self.embed_dim = embed_dim

        self.proj = nn.Conv2D(
            in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)
        if norm_layer is not None:
            self.norm = norm_layer(embed_dim)
        else:
            self.norm = None

    def forward(self, x):
        """Forward function."""
        # padding
        _, _, H, W = x.shape
        if W % self.patch_size[1] != 0:
            x = F.pad(x, [0, self.patch_size[1] - W % self.patch_size[1], 0, 0])
        if H % self.patch_size[0] != 0:
            x = F.pad(x, [0, 0, 0, self.patch_size[0] - H % self.patch_size[0]])

        x = self.proj(x)  # B C Wh Ww
        if self.norm is not None:
            _, _, Wh, Ww = x.shape
            x = x.flatten(2).transpose([0, 2, 1])
            x = self.norm(x)
            x = x.transpose([0, 2, 1]).reshape([-1, self.embed_dim, Wh, Ww])

        return x


class SwinTransformer(nn.Layer):
    """
    The SwinTransformer implementation based on PaddlePaddle.
    The original article refers to
    Liu, Ze, et al. "Swin Transformer: Hierarchical Vision Transformer using Shifted Windows"
    (https://arxiv.org/abs/2103.14030)
    Args:
        pretrain_img_size (int): Input image size for training the pretrained model, used in absolute postion embedding. Default: 224.
        patch_size (int | tuple(int)): Patch size. Default: 4.
        in_chans (int): Number of input image channels. Default: 3.
        embed_dim (int): Number of linear projection output channels. Default: 96.
        depths (tuple[int]): Depths of each Swin Transformer stage.
        num_heads (tuple[int]): Number of attention head of each stage.
        window_size (int): Window size. Default: 7.
        mlp_ratio (float): Ratio of mlp hidden dim to embedding dim. Default: 4.
        qkv_bias (bool): If True, add a learnable bias to query, key, value. Default: True
        qk_scale (float): Override default qk scale of head_dim ** -0.5 if set.
        drop_rate (float): Dropout rate.
        attn_drop_rate (float): Attention dropout rate. Default: 0.
        drop_path_rate (float): Stochastic depth rate. Default: 0.2.
        norm_layer (nn.Layer): Normalization layer. Default: nn.LayerNorm.
        ape (bool): If True, add absolute position embedding to the patch embedding. Default: False.
        patch_norm (bool): If True, add normalization after patch embedding. Default: True.
        out_indices (Sequence[int]): Output from which stages.
        frozen_stages (int): Stages to be frozen (stop grad and set eval mode). -1 means not freezing any parameters. Default: -1.
        pretrained (str, optional): The path or url of pretrained model. Default: None.
    """

    def __init__(self,
                 pretrain_img_size=224,
                 patch_size=4,
                 in_chans=3,
                 embed_dim=96,
                 depths=[2, 2, 6, 2],
                 num_heads=[3, 6, 12, 24],
                 window_size=7,
                 mlp_ratio=4.,
                 qkv_bias=True,
                 qk_scale=None,
                 drop_rate=0.,
                 attn_drop_rate=0.,
                 drop_path_rate=0.2,
                 norm_layer=nn.LayerNorm,
                 ape=False,
                 patch_norm=True,
                 out_indices=(0, 1, 2, 3),
                 frozen_stages=-1,
                 pretrained=None):
        super().__init__()

        self.pretrain_img_size = pretrain_img_size
        self.num_layers = len(depths)
        self.embed_dim = embed_dim
        self.ape = ape
        self.patch_norm = patch_norm
        self.out_indices = out_indices
        self.frozen_stages = frozen_stages

        # split image into non-overlapping patches
        self.patch_embed = PatchEmbed(
            patch_size=patch_size,
            in_chans=in_chans,
            embed_dim=embed_dim,
            norm_layer=norm_layer if self.patch_norm else None)
        # self.patch_embed = nn.Conv2D(3, 96, 7, 1, 3)

        # absolute position embedding
        if self.ape:
            pretrain_img_size = to_2tuple(pretrain_img_size)
            patch_size = to_2tuple(patch_size)
            patches_resolution = [
                pretrain_img_size[0] // patch_size[0],
                pretrain_img_size[1] // patch_size[1]
            ]

            self.absolute_pos_embed = self.create_parameter(
                shape=(1, embed_dim, patches_resolution[0],
                       patches_resolution[1]),
                default_initializer=zeros_)
            self.add_parameter("absolute_pos_embed", self.absolute_pos_embed)
            trunc_normal_(self.absolute_pos_embed)

        self.pos_drop = nn.Dropout(p=drop_rate)

        # stochastic depth
        dpr = np.linspace(0, drop_path_rate, sum(depths)).tolist()

        # build layers
        self.layers = nn.LayerList()
        for i_layer in range(self.num_layers):
            layer = BasicLayer(
                dim=int(embed_dim * 2**i_layer),
                depth=depths[i_layer],
                num_heads=num_heads[i_layer],
                window_size=window_size,
                mlp_ratio=mlp_ratio,
                qkv_bias=qkv_bias,
                qk_scale=qk_scale,
                drop=drop_rate,
                attn_drop=attn_drop_rate,
                drop_path=dpr[sum(depths[:i_layer]):sum(depths[:i_layer + 1])],
                norm_layer=norm_layer,
                downsample=PatchMerging
                if (i_layer < self.num_layers - 1) else None)
            self.layers.append(layer)

        feat_channels = [int(embed_dim * 2**i) for i in range(self.num_layers)]
        self.feat_channels = feat_channels

        # add a norm layer for each output
        for i_layer in out_indices:
            layer = norm_layer(feat_channels[i_layer])
            layer_name = f'norm{i_layer}'
            self.add_sublayer(layer_name, layer)

        self._freeze_stages()

        self.pretrained = pretrained
        self.init_weights(self.pretrained)

    def _freeze_stages(self):
        if self.frozen_stages >= 0:
            self.patch_embed.eval()
            for param in self.patch_embed.parameters():
                param.requires_grad = False

        if self.frozen_stages >= 1 and self.ape:
            self.absolute_pos_embed.requires_grad = False

        if self.frozen_stages >= 2:
            self.pos_drop.eval()
            for i in range(0, self.frozen_stages - 1):
                layer = self.layers[i]
                layer.eval()
                for param in layer.parameters():
                    param.requires_grad = False

    def init_weights(self, pretrained=None):
        """Initialize the weights in backbone.
        Args:
            pretrained (str, optional): Path to pre-trained weights.
                Defaults to None.
        """
        if pretrained is not None:
            load_pretrained_model(self, self.pretrained)
        else:
            for sublayer in self.sublayers():
                if isinstance(sublayer, nn.Linear):
                    trunc_normal_(sublayer.weight)
                    if isinstance(sublayer,
                                  nn.Linear) and sublayer.bias is not None:
                        zeros_(sublayer.bias)
                elif isinstance(sublayer, nn.LayerNorm):
                    zeros_(sublayer.bias)
                    ones_(sublayer.weight)

    def forward(self, x):
        """Forward function."""
        x = self.patch_embed(x)

        _, _, Wh, Ww = x.shape
        if self.ape:
            # interpolate the position embedding to the corresponding size
            absolute_pos_embed = F.interpolate(
                self.absolute_pos_embed, size=(Wh, Ww), mode='bicubic')
            x = (x + absolute_pos_embed).flatten(2).transpose(1, 2)  # B Wh*Ww C
        else:
            x = x.flatten(2).transpose([0, 2, 1])
        x = self.pos_drop(x)

        outs = []
        for i in range(self.num_layers):
            layer = self.layers[i]
            x_out, H, W, x, Wh, Ww = layer(x, Wh, Ww)

            if i in self.out_indices:
                norm_layer = getattr(self, f'norm{i}')
                x_out = norm_layer(x_out)

                out = x_out.reshape(
                    [-1, H, W, self.feat_channels[i]]).transpose([0, 3, 1, 2])
                outs.append(out)

        return tuple(outs)

    # def train(self):
    #     """Convert the model into training mode while keep layers freezed."""
    #     super(SwinTransformer, self).train()
    #     self._freeze_stages()

class FPNBlock(nn.Layer):
    def __init__(self, pyramid_channels, skip_channels):
        super().__init__()
        self.skip_conv = nn.Conv2D(skip_channels, pyramid_channels, kernel_size=1)

    def forward(self, x, skip=None):
        x = F.interpolate(x, scale_factor=2, mode="nearest")
        skip = self.skip_conv(skip)
        x = x + skip
        return x

class Conv3x3GNReLU(nn.Layer):
    def __init__(self, in_channels, out_channels, upsample=False):
        super().__init__()
        self.upsample = upsample
        self.block = nn.Sequential(
            nn.Conv2D(
                in_channels, out_channels, (3, 3), stride=1, padding=1, bias_attr=False
            ),
            nn.GroupNorm(32, out_channels),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.block(x)
        if self.upsample:
            x = F.interpolate(x, scale_factor=2, mode="bilinear", align_corners=True)
        return x

class SegmentationBlock(nn.Layer):
    def __init__(self, in_channels, out_channels, n_upsamples=0):
        super().__init__()

        blocks = [Conv3x3GNReLU(in_channels, out_channels, upsample=bool(n_upsamples))]

        if n_upsamples > 1:
            for _ in range(1, n_upsamples):
                blocks.append(Conv3x3GNReLU(out_channels, out_channels, upsample=True))

        self.block = nn.Sequential(*blocks)

    def forward(self, x):
        return self.block(x)

class MergeBlock(nn.Layer):
    def __init__(self, policy):
        super().__init__()
        if policy not in ["add", "cat"]:
            raise ValueError(
                "`merge_policy` must be one of: ['add', 'cat'], got {}".format(
                    policy
                )
            )
        self.policy = policy

    def forward(self, x):
        if self.policy == 'add':
            return sum(x)
        elif self.policy == 'cat':
            return paddle.concat(x, axis=1)
        else:
            raise ValueError(
                "`merge_policy` must be one of: ['add', 'cat'], got {}".format(self.policy)
            )


class FPNDecoder(nn.Layer):
    def __init__(
            self,
            encoder_channels,
            encoder_depth=5,
            pyramid_channels=256,
            segmentation_channels=128,
            dropout=0.1,
            merge_policy="add",
    ):
        super().__init__()

        self.out_channels = segmentation_channels if merge_policy == "add" else segmentation_channels * 4
        if encoder_depth < 3:
            raise ValueError("Encoder depth for FPN decoder cannot be less than 3, got {}.".format(encoder_depth))

        encoder_channels = encoder_channels[::-1]
        encoder_channels = encoder_channels[:encoder_depth + 1]

        self.p5 = nn.Conv2D(encoder_channels[0], pyramid_channels, kernel_size=1)
        self.p4 = FPNBlock(pyramid_channels, encoder_channels[1])
        self.p3 = FPNBlock(pyramid_channels, encoder_channels[2])
        self.p2 = FPNBlock(pyramid_channels, encoder_channels[3])

        self.seg_blocks = nn.LayerList([
            SegmentationBlock(pyramid_channels, segmentation_channels, n_upsamples=n_upsamples)
            for n_upsamples in [3, 2, 1, 0]
        ])

        self.merge = MergeBlock(merge_policy)
        self.dropout = nn.Dropout2D(p=dropout)

    def forward(self, *features):
        c2, c3, c4, c5 = features[-4:]

        p5 = self.p5(c5)
        p4 = self.p4(p5, c4)
        p3 = self.p3(p4, c3)
        p2 = self.p2(p3, c2)

        feature_pyramid = [seg_block(p) for seg_block, p in zip(self.seg_blocks, [p5, p4, p3, p2])]
        x = self.merge(feature_pyramid)
        x = self.dropout(x)

        return x


class swimb_fpn(nn.Layer):
    def __init__(self,
                 encoder='resnet34',
                 encoder_depth=4,
                 in_channels=3,
                 pretrained=True,
                 classes=2,
                 decoder_pyramid_channels: int = 256,
                 decoder_segmentation_channels: int = 128,
                 decoder_merge_policy: str = "add",
                 decoder_dropout: float = 0.2,
                 pretrain=None,
                 ):
        super(swimb_fpn, self).__init__()

        # ENCODER
        self.encoder = SwinTransformer(
        pretrain_img_size=384,
        embed_dim=128,
        depths=[2, 2, 18, 2],
        num_heads=[4, 8, 16, 32],
        window_size=12,
        pretrained=pretrain
        )
        encoder_channels = [128, 256, 512, 1024]


        # DECODER
        self.decoder = FPNDecoder(
            encoder_channels=encoder_channels,
            encoder_depth=encoder_depth,
            pyramid_channels=decoder_pyramid_channels,
            segmentation_channels=decoder_segmentation_channels,
            dropout=decoder_dropout,
            merge_policy=decoder_merge_policy,
        )

        # PREDICT
        self.pred_head = nn.Conv2D(decoder_segmentation_channels, classes, 3, 1, 1)
        # self.up = nn.Conv2DTranspose(in_channels=256,
        #                              out_channels=128,
        #                              kernel_size=2,
        #                              stride=2,
        #                              padding=0)
        # self.conv1 = nn.Conv2D(in_channels=128,
        #                        out_channels=64,
        #                        kernel_size=3,
        #                        stride=1,
        #                        padding=1)
        # self.bn = nn.BatchNorm(64, act="relu")
        # self.up2 = nn.Conv2DTranspose(in_channels=64,
        #                              out_channels=classes,
        #                              kernel_size=2,
        #                              stride=2,
        #                              padding=0)
        # self.conv2 = nn.Conv2D(in_channels=classes,
        #                        out_channels=classes,
        #                        kernel_size=31,
        #                        stride=1,
        #                        padding=15)

    def forward(self, x):
        encoder_feats = self.encoder(x)

        decoder_feat = self.decoder(*encoder_feats)
        out = self.pred_head(decoder_feat)
        out = F.interpolate(out, size=x.shape[-2:], mode='bilinear', align_corners=False)
        # out = self.up(out)
        # out = self.bn(self.conv1(out))
        # out = self.up2(out)
        # out = self.conv2(out)

        return out




定义loss函数

In [ ]:
import paddle
from paddle import nn
import paddle.nn.functional as F

# from paddleseg.cvlibs import manager


# @manager.LOSSES.add_component
class DiceLoss(nn.Layer):
    """
    The implements of the dice loss.

    Args:
        weight (list[float], optional): The weight for each class. Default: None.
        ignore_index (int64): ignore_index (int64, optional): Specifies a target value that
            is ignored and does not contribute to the input gradient. Default ``255``.
        smooth (float32): Laplace smoothing to smooth dice loss and accelerate convergence.
            Default: 1.0
    """

    def __init__(self, weight=None, ignore_index=255, smooth=1.0):
        super().__init__()
        self.weight = weight
        self.ignore_index = ignore_index
        self.smooth = smooth
        self.eps = 1e-8

    def forward(self, logits, labels):
        num_class = logits.shape[1]
        if self.weight is not None:
            assert num_class == len(self.weight), \
                "The lenght of weight should be euqal to the num class"

        logits = F.softmax(logits, axis=1)
        labels_one_hot = F.one_hot(labels, num_class)
        labels_one_hot = paddle.transpose(labels_one_hot, [0, 3, 1, 2])

        mask = labels != self.ignore_index
        mask = paddle.cast(paddle.unsqueeze(mask, 1), 'float32')

        dice_loss = 0.0
        for i in range(num_class):
            dice_loss_i = dice_loss_helper(logits[:, i], labels_one_hot[:, i],
                                           mask, self.smooth, self.eps)
            if self.weight is not None:
                dice_loss_i *= self.weight[i]
            dice_loss += dice_loss_i
        dice_loss = dice_loss / num_class

        return dice_loss


def dice_loss_helper(logit, label, mask, smooth, eps):
    assert logit.shape == label.shape, \
        "The shape of logit and label should be the same"
    logit = paddle.reshape(logit, [0, -1])
    label = paddle.reshape(label, [0, -1])
    mask = paddle.reshape(mask, [0, -1])
    logit *= mask
    label *= mask
    intersection = paddle.sum(logit * label, axis=1)
    cardinality = paddle.sum(logit + label, axis=1)
    dice_loss = 1 - (2 * intersection + smooth) / (cardinality + smooth + eps)
    dice_loss = dice_loss.mean()
    return dice_loss


from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import paddle
from paddle import nn
import paddle.nn.functional as F

# from paddleseg.cvlibs import manager


# @manager.LOSSES.add_component
class LovaszSoftmaxLoss(nn.Layer):
    """
    Multi-class Lovasz-Softmax loss.

    Args:
        ignore_index (int64): Specifies a target value that is ignored and does not contribute to the input gradient. Default ``255``.
        classes (str|list): 'all' for all, 'present' for classes present in labels, or a list of classes to average.
    """

    def __init__(self, ignore_index=255, classes='present'):
        super(LovaszSoftmaxLoss, self).__init__()
        self.ignore_index = ignore_index
        self.classes = classes

    def forward(self, logits, labels):
        r"""
        Forward computation.

        Args:
            logits (Tensor): Shape is [N, C, H, W], logits at each prediction (between -\infty and +\infty).
            labels (Tensor): Shape is [N, 1, H, W] or [N, H, W], ground truth labels (between 0 and C - 1).
        """
        probas = F.softmax(logits, axis=1)
        vprobas, vlabels = flatten_probas(probas, labels, self.ignore_index)
        loss = lovasz_softmax_flat(vprobas, vlabels, classes=self.classes)
        return loss


# @manager.LOSSES.add_component
class LovaszHingeLoss(nn.Layer):
    """
    Binary Lovasz hinge loss.

    Args:
        ignore_index (int64): Specifies a target value that is ignored and does not contribute to the input gradient. Default ``255``.
    """

    def __init__(self, ignore_index=255):
        super(LovaszHingeLoss, self).__init__()
        self.ignore_index = ignore_index

    def forward(self, logits, labels):
        r"""
        Forward computation.

        Args:
            logits (Tensor): Shape is [N, 1, H, W] or [N, 2, H, W], logits at each pixel (between -\infty and +\infty).
            labels (Tensor): Shape is [N, 1, H, W] or [N, H, W], binary ground truth masks (0 or 1).
        """
        if logits.shape[1] == 2:
            logits = binary_channel_to_unary(logits)
        loss = lovasz_hinge_flat(
            *flatten_binary_scores(logits, labels, self.ignore_index))
        return loss


def lovasz_grad(gt_sorted):
    """
    Computes gradient of the Lovasz extension w.r.t sorted errors.
    See Alg. 1 in paper.
    """
    gts = paddle.sum(gt_sorted)
    p = len(gt_sorted)

    intersection = gts - paddle.cumsum(gt_sorted, axis=0)
    union = gts + paddle.cumsum(1 - gt_sorted, axis=0)
    jaccard = 1.0 - intersection.cast('float32') / union.cast('float32')

    if p > 1:  # cover 1-pixel case
        jaccard[1:p] = jaccard[1:p] - jaccard[0:-1]
    return jaccard


def binary_channel_to_unary(logits, eps=1e-9):
    """
    Converts binary channel logits to unary channel logits for lovasz hinge loss.
    """
    probas = F.softmax(logits, axis=1)
    probas = probas[:, 1, :, :]
    logits = paddle.log(probas + eps / (1 - probas + eps))
    logits = logits.unsqueeze(1)
    return logits


def lovasz_hinge_flat(logits, labels):
    r"""
    Binary Lovasz hinge loss.

    Args:
        logits (Tensor): Shape is [P], logits at each prediction (between -\infty and +\infty).
        labels (Tensor): Shape is [P], binary ground truth labels (0 or 1).
    """
    if len(labels) == 0:
        # only void pixels, the gradients should be 0
        return logits.sum() * 0.
    signs = 2. * labels - 1.
    signs.stop_gradient = True
    errors = 1. - logits * signs
    errors_sorted, perm = paddle.fluid.core.ops.argsort(errors, 'axis', 0,
                                                        'descending', True)
    errors_sorted.stop_gradient = False
    gt_sorted = paddle.gather(labels, perm)
    grad = lovasz_grad(gt_sorted)
    grad.stop_gradient = True
    loss = paddle.sum(F.relu(errors_sorted) * grad)
    return loss


def flatten_binary_scores(scores, labels, ignore=None):
    """
    Flattens predictions in the batch (binary case).
    Remove labels according to 'ignore'.
    """
    scores = paddle.reshape(scores, [-1])
    labels = paddle.reshape(labels, [-1])
    labels.stop_gradient = True
    if ignore is None:
        return scores, labels
    valid = labels != ignore
    valid_mask = paddle.reshape(valid, (-1, 1))
    indexs = paddle.nonzero(valid_mask)
    indexs.stop_gradient = True
    vscores = paddle.gather(scores, indexs[:, 0])
    vlabels = paddle.gather(labels, indexs[:, 0])
    return vscores, vlabels


def lovasz_softmax_flat(probas, labels, classes='present'):
    """
    Multi-class Lovasz-Softmax loss.

    Args:
        probas (Tensor): Shape is [P, C], class probabilities at each prediction (between 0 and 1).
        labels (Tensor): Shape is [P], ground truth labels (between 0 and C - 1).
        classes (str|list): 'all' for all, 'present' for classes present in labels, or a list of classes to average.
    """
    if probas.numel() == 0:
        # only void pixels, the gradients should be 0
        return probas * 0.
    C = probas.shape[1]
    losses = []
    classes_to_sum = list(range(C)) if classes in ['all', 'present'
                                                   ] else classes
    for c in classes_to_sum:
        fg = paddle.cast(labels == c, probas.dtype)  # foreground for class c
        if classes == 'present' and fg.sum() == 0:
            continue
        fg.stop_gradient = True
        if C == 1:
            if len(classes_to_sum) > 1:
                raise ValueError('Sigmoid output possible only with 1 class')
            class_pred = probas[:, 0]
        else:
            class_pred = probas[:, c]
        errors = paddle.abs(fg - class_pred)
        errors_sorted, perm = paddle.fluid.core.ops.argsort(errors, 'axis', 0,
                                                            'descending', True)
        errors_sorted.stop_gradient = False

        fg_sorted = paddle.gather(fg, perm)
        fg_sorted.stop_gradient = True

        grad = lovasz_grad(fg_sorted)
        grad.stop_gradient = True
        loss = paddle.sum(errors_sorted * grad)
        losses.append(loss)

    if len(classes_to_sum) == 1:
        return losses[0]

    losses_tensor = paddle.stack(losses)
    mean_loss = paddle.mean(losses_tensor)
    return mean_loss


def flatten_probas(probas, labels, ignore=None):
    """
    Flattens predictions in the batch.
    """
    if len(probas.shape) == 3:
        probas = paddle.unsqueeze(probas, axis=1)
    C = probas.shape[1]
    probas = paddle.transpose(probas, [0, 2, 3, 1])
    probas = paddle.reshape(probas, [-1, C])
    labels = paddle.reshape(labels, [-1])
    if ignore is None:
        return probas, labels
    valid = labels != ignore
    valid_mask = paddle.reshape(valid, [-1, 1])
    indexs = paddle.nonzero(valid_mask)
    indexs.stop_gradient = True
    vprobas = paddle.gather(probas, indexs[:, 0])
    vlabels = paddle.gather(labels, indexs[:, 0])
    return vprobas, vlabels


对网络进行训练

In [ ]:
import paddle
import os
import time


os.environ["CUDA_VISIBLE_DEVICES"] = '0'
paddle.set_device('gpu')
batch_s = 4
output_model_dir = './swimb'
if not os.path.exists(output_model_dir):
    os.mkdir(output_model_dir)

# log file
file_path = './swimb/train_log.txt'
f_log = open(file_path, 'a')

######## model
# model = get_unetrw()
# model = get_swiml()
model = swimb_fpn(pretrain=r'D:\yyc\yycpython\Document_seg\model\SwinTransformer_base_patch4_window12_384_22kto1k_pretrained.pdparams')
model.train()

train_dataset=MyDateset()
train_dataloader = paddle.io.DataLoader(
    train_dataset,
    batch_size=batch_s,
    shuffle=True,
    drop_last=False)


max_epoch=260
scheduler = paddle.optimizer.lr.CosineAnnealingDecay(learning_rate=0.001, T_max=max_epoch)
opt = paddle.optimizer.Adam(learning_rate=scheduler, parameters=model.parameters())

loss_lvz = LovaszSoftmaxLoss()
loss_dice = DiceLoss()

start_epoch = 0
epoch_steps = train_dataset.__len__()//batch_s
print('epoch_steps', epoch_steps)
now_step = start_epoch*epoch_steps

for epoch in range(start_epoch, max_epoch):
    f_log.write('epoch:'+str(epoch)+'\n')
    for step, data in enumerate(train_dataloader):
        now_step+=1

        img, label = data

        pre = model(img)

        # loss = softmax_with_cross_entropy(paddle.transpose(pre, [0, 2, 3, 1]), label_sig.unsqueeze(3))
        loss1 = loss_lvz(pre, label)
        loss2 = loss_dice(pre, label)

        loss = 0.65 * loss1 + loss2 * 0.35

        loss.backward()
        opt.step()
        opt.clear_gradients()

        scheduler.step(epoch + step/epoch_steps)

        lr_update = opt.get_lr()
        if now_step%10==0:
            content_log = time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())) + ' ' + "epoch: {}, batch: {}, loss is: {}, lossohem is: {},lr:{}".format(epoch, step, ((loss1 + loss2) * 0.5).mean().numpy(), loss.mean().numpy(),lr_update)
            print(content_log)
            f_log.write(content_log + '\n')
    paddle.save(model.state_dict(), os.path.join(output_model_dir, 'model_' + str(epoch)+'.pdparams'))
f_log.close()
paddle.save(model.state_dict(), 'model.pdparams')

对测试集进行推理

In [ ]:
import os
import sys
import glob
import json
import cv2
import paddle
import numpy as np

def process(src_image_dir, save_dir):

    model = swimb_fpn()
    model_state_dict = paddle.load('./model/model_137.pdparams')
    model.set_state_dict(model_state_dict)
    model.eval()

    # json_results = []

    image_paths = glob.glob(os.path.join(src_image_dir, "*.jpg"))
    for image_path in image_paths:
        # do something
        img = cv2.imread(image_path)
        h, w, c = img.shape
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = paddle.vision.transforms.resize(img, (512, 512), interpolation='bilinear')
        img = img.transpose((2, 0, 1))
        img = img / 255
        img = paddle.to_tensor(img).astype('float32')
        img = img.unsqueeze(0)
        with paddle.no_grad():
            s_predict_seg = model(img)

            s_image_h = paddle.flip(img, axis=[3])
            s_predict_h = model(s_image_h)
            s_predict_h_seg = paddle.flip(s_predict_h, axis=[3])

            s_image_v = paddle.flip(img, axis=[2])
            s_predict_v = model(s_image_v)
            s_predict_v_seg = paddle.flip(s_predict_v, axis=[2])

            pre = 0.5 * s_predict_seg + 0.25 * s_predict_h_seg + 0.25 * s_predict_v_seg

        # 保存结果图片
        out_image = paddle.argmax(pre, axis=1).astype(float)
        # print(out_image[0,200:300,200:300])
        out_image[out_image == 1] = 255
        out_image = paddle.fluid.layers.expand(out_image, expand_times=[3, 1, 1])
        # print(out_image[0,200:300,200:300])
        out_image = out_image.transpose((1, 2, 0)).numpy().astype(np.uint8)

        out_image = paddle.vision.transforms.resize(out_image, (h, w), interpolation='bilinear')


        save_path = os.path.join(save_dir, os.path.basename(image_path).replace(".jpg", ".png"))
        cv2.imwrite(save_path, out_image)

src_image_dir = r'D:\yyc\competition\AIstudio\Document_detection\document_detection_testB_dataset'
save_dir = r'D:\yyc\competition\AIstudio\Document_detection\results'

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

process(src_image_dir, save_dir)

